In [73]:
import pandas as pd
from scipy.stats import pearsonr

In [2]:
df_titanic = pd.read_csv('./data/titanic.csv')

In [3]:
df_titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [84]:
lista_numericas = []
for column in df_titanic.columns:
    
    if df_titanic[column].dtypes in ['int8', 'int16', 'int32','int64', 'float16', 'float32', 'float64']:
        lista_numericas.append(column)

print(lista_numericas)

for columna in lista_numericas:

    no_nulos = df_titanic.dropna(subset= ['fare', columna])
    corr, pearson = pearsonr(no_nulos['fare'], no_nulos[columna])
    print(corr, pearson)

'''
correlaciones = df_titanic[lista_numericas].corr()

correlaciones
'''

['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare']
          survived    pclass       age     sibsp     parch      fare
survived  1.000000 -0.338481 -0.077221 -0.035322  0.081629  0.257307
pclass   -0.338481  1.000000 -0.369226  0.083081  0.018443 -0.549500
age      -0.077221 -0.369226  1.000000 -0.308247 -0.189119  0.096067
sibsp    -0.035322  0.083081 -0.308247  1.000000  0.414838  0.159651
parch     0.081629  0.018443 -0.189119  0.414838  1.000000  0.216225
fare      0.257307 -0.549500  0.096067  0.159651  0.216225  1.000000
0.25730652238496243 6.1201893419246185e-15
-0.5494996199439077 1.9673861734225787e-71
0.09606669176903891 0.010216277504447006
0.15965104324216098 1.6712557684699932e-06
0.2162249447707648 6.915291577876966e-11
0.9999999999999996 0.0


'\ncorrelaciones = df_titanic[lista_numericas].corr()\n\ncorrelaciones\n'

In [102]:
type(df_titanic['fare'])

pandas.core.series.Series

In [96]:
def get_features_num_regresion(dataframe, target_col, umbral_corr, pvalue= None):

    '''
    La funcion get_features_num_regresion devuelve las features para la creacion de un modelo de machine learning.

    Estas features deben ser variables numericas y disponer de una correlacón y significacion estadistica significativa con el target, definidos previamente por el usuario.
    La significacion estadistica es nula por defecto.

    Argumentos:

    .-dataframe(pandas.core.frame.DataFrame) -> un dataframe pandas sobre el que realizar el estudio
    .-target_col(str) -> la columna seleccionada como target para nuestro modelo
    .-umbral_corr(float) -> la correlacion minima exigida a una variable con el target para ser designado como feature. Debe estar comprendido entre 0 y 1
    .-pvalue(float) -> la significacion estadistica Pearson maxima exigida a una variable para ser designada como feature (generalmente 0.005). None por defecto

    Retorna:

    Lista con las columnas designadas como features para el modelo.
    Tipo lista compuesto por cadenas de texto.
    '''

    cardinalidad = dataframe[target_col].nunique() / len(dataframe[target_col])

    if (type(umbral_corr) != float) or (umbral_corr < 0) or (umbral_corr > 1):

        print('Variable umbral_corr incorrecto.')
        return None

    elif dataframe[target_col].dtype not in ['int8', 'int16', 'int32','int64', 'float16', 'float32', 'float64']:

        print('La columna seleccionada como target debe ser numerica.')
        return None
    
    elif cardinalidad < 0: # este no se si ponerlo

        print('Tu variable target tiene una cardinalidad muy baja para ser target.')
        return None
    
    lista_numericas = []
    for column in dataframe.columns:
        
        if dataframe[column].dtypes in ['int8', 'int16', 'int32','int64', 'float16', 'float32', 'float64']:
            lista_numericas.append(column)

    lista_numericas.remove(target_col)
    lista_features = []
    for columna in lista_numericas:

        no_nulos = dataframe.dropna(subset= [target_col, columna])
        corr, pearson = pearsonr(no_nulos[target_col], no_nulos[columna])

        if pvalue != None:
            if (abs(corr) >= umbral_corr) and (pearson <= pvalue):
                lista_features.append(columna)
        else:
            if abs(corr) >= umbral_corr:
                lista_features.append(columna)
    
    return lista_features

In [100]:
get_features_num_regresion(df_titanic, 'fare', 0.2)

['survived', 'pclass', 'parch']